### Подключение библиотек

In [1]:
import json
import networkx as nx
from collections import defaultdict

### Загрузка обменного файла STEP (JSON)

In [2]:
# Открываем JSON-файл
with open("STEP_product_structure_example.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Создаем пустой граф
graph = nx.DiGraph()

### Добавление всех объектов в массив, добавление нужных узлов в GraphML

In [3]:
# Создаем словарь для поиска узлов по их id
nodes = {}

# Заполняем граф узлами и создаем соответствующие атрибуты
for instance in data.get("instances", []):
    node_id = instance["id"]
    attributes = instance.get("attributes", {})
    entity_type = instance.get("type", "")
    nodes[node_id]= attributes
    if entity_type== 'product_definition':
        prd_name=''
        attr_version_value = attributes.get("id", "")
        formation_ref= attributes.get("formation", "")
        if formation_ref:
            formation_by_ref= nodes.get(formation_ref, "")
            if formation_by_ref:
                prd_ref= formation_by_ref.get('of_product', "")
                if prd_ref:
                    prd_by_ref= nodes.get(prd_ref, "")
                    if prd_by_ref:
                        attr_id_value = prd_by_ref.get("id", "")
                        attr_name_value = prd_by_ref.get("name", "")
                        prd_name=attr_id_value
                        if attr_name_value:
                            prd_name= f'{attr_id_value} :: {attr_name_value}'
                        if attr_version_value:
                            prd_name+=f' (версия {attr_version_value})'
        print(f'Добавлен узел (изделие) с id: {node_id} и label: {prd_name}')
        graph.add_node(node_id, label=prd_name, shape = "ellipse")
    elif entity_type== 'organization':
        org_name=''
        attr_id_value = attributes.get("id", "")
        attr_name_value = attributes.get("name", "")
        org_name=attr_id_value
        if attr_name_value:
            org_name= f'{attr_id_value} :: {attr_name_value}'
        print(f'Добавлен узел (организация) с id: {node_id} и label: {org_name}')
        graph.add_node(node_id, label=org_name, shape="rectangle")

Добавлен узел (изделие) с id: #3 и label: АБВГ.123456.001 :: Прижим (версия EPS001)
Добавлен узел (изделие) с id: #7 и label: АБВГ.123456.001-01 :: Набор стандартных изделий  (версия EPS004)
Добавлен узел (изделие) с id: #11 и label:  Трубка ТВ-40,3, белая, 1 сорта ГОСТ 19034-82 :: Трубка (версия EPS008)
Добавлен узел (изделие) с id: #17 и label:  Шайба 12.01.08кп.016ГОСТ10450-78 :: Шайба (версия 001)
Добавлен узел (изделие) с id: #22 и label: АБВГ.715121.006 :: Ось (версия EPS005)
Добавлен узел (изделие) с id: #29 и label: ЕКУЦ.301527.001 :: Петля (версия EPS003)
Добавлен узел (организация) с id: #41 и label: ЕКУЦ :: АО «Организация002»


### Добавление связей между изделиями в GraphML как ребра графа

In [4]:
levels={}
def extract_product_structure_data(attributes):
    position = attributes.get("reference_designator", "<не задана>")
    parent_id = attributes["relating_product_definition"]
    child_id = attributes["related_product_definition"]
    quantity_ref = attributes.get("quantity")
    
    # Определяем количество элементов, если есть ссылка
    quantity = "?"
    if quantity_ref and isinstance(quantity_ref, str) and quantity_ref.startswith("#"):
        # Если quantity - это ссылка на другой объект
        quantity="<не задано>"
        quantity_by_ref = nodes.get(quantity_ref, "")
        if quantity_by_ref:
            quantity_value= quantity_by_ref.get('value_component', "-1")
            unit_ref= quantity_by_ref.get('unit_component', "")
            quantity_uom_unit=None
            if unit_ref:
                unit_by_ref= nodes.get(unit_ref, "")
                if unit_by_ref:
                    quantity_uom_unit= unit_by_ref.get("id", "")
            quantity= quantity_value 
            if quantity_uom_unit:
                quantity= f'{quantity_value} {quantity_uom_unit}'
    elif quantity_ref:
        # Если quantity - это непосредственно значение
        quantity = quantity_ref
    
    # Добавляем ребро с подписью количества
    label= f'Состоит из, поз.{position}, кол-во: {quantity}'
    if not position:
        label=f'Состоит из, кол-во: {quantity}'
    print(f'Добавлено ребро графа (связь между изделиями) с parent_id={parent_id}, child_id={child_id}, label={label}')
    graph.add_edge(parent_id, child_id, label=label)    

    # Определяем уровни узлов
    if parent_id not in levels:
        levels[parent_id] = 0  # Корневой узел
    levels[child_id] = levels[parent_id] + 1

# Обрабатываем связи между изделиями, количество
for instance in data.get("instances", []):
    attributes = instance.get("attributes", {})
    if "relating_product_definition" in attributes:
        extract_product_structure_data(attributes)

Добавлено ребро графа (связь между изделиями) с parent_id=#3, child_id=#7, label=Состоит из, кол-во: 1 шт.
Добавлено ребро графа (связь между изделиями) с parent_id=#7, child_id=#11, label=Состоит из, поз.5, кол-во: 0,4 м.
Добавлено ребро графа (связь между изделиями) с parent_id=#7, child_id=#17, label=Состоит из, поз.3, кол-во: 4 шт.
Добавлено ребро графа (связь между изделиями) с parent_id=#3, child_id=#22, label=Состоит из, поз.2, кол-во: 1 шт.
Добавлено ребро графа (связь между изделиями) с parent_id=#3, child_id=#29, label=Состоит из, поз.1, кол-во: 1 шт.


### Добавление организаций и их связей с изделиями

In [5]:
def extract_organizations_data(attributes):
    parent_id = attributes["assigned_product"]
    child_id = attributes["assigned_organization"]
    role = "?"
    role_ref = attributes.get("role", "")
    if role_ref:
        role_by_ref= nodes.get(role_ref, "")
        if role_by_ref:
            role= role_by_ref.get("name", "?")
    
    # Добавляем ребро со ссылкой на организацию
    print(f'Добавлено ребро графа (связь с организацией) с parent_id={parent_id}, child_id={child_id}, label={role}')
    graph.add_edge(parent_id, child_id, label=role) 

    # Определяем уровни узлов
    if parent_id not in levels:
        levels[parent_id] = 0  # Корневой узел
    levels[child_id] = levels[parent_id] + 1

# Обрабатываем связи между изделиями, количество
for instance in data.get("instances", []):
    attributes = instance.get("attributes", {})
    entity_type = instance.get("type", "")
    if entity_type=="eskd_organization_product_assignment":
        extract_organizations_data(attributes)

Добавлено ребро графа (связь с организацией) с parent_id=#29, child_id=#41, label=Разработчик


### Сохранение файла GraphML

In [6]:
# Добавляем уровень иерархии в узлы графа
for node, level in levels.items():
    graph.nodes[node]["hierarchical_level"] =str(level)
# сохраняем файл GraphML
nx.write_graphml(graph, "example_product_structure.graphml")

print("Граф успешно сохранён в example_product_structure.graphml")

Граф успешно сохранён в example_product_structure.graphml
